<a href="https://colab.research.google.com/github/GiammarcoBozzelli/DSML/blob/main/20250517_Camembert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#imoprt packages
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import nltk
nltk.download('stopwords')
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from transformers import CamembertTokenizer, CamembertModel, pipeline
import torch

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load the dataset
url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv"
data = pd.read_csv(url)

   id                                           sentence difficulty
0   0  Les coûts kilométriques réels peuvent diverger...         C1
1   1  Le bleu, c'est ma couleur préférée mais je n'a...         A1
2   2  Le test de niveau en français est sur le site ...         A1
3   3           Est-ce que ton mari est aussi de Boston?         A1
4   4  Dans les écoles de commerce, dans les couloirs...         B1


In [ ]:
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

# Load your dataset
df = pd.read_csv('https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/training_data.csv')  # Update this with the correct path to your dataset

# Install necessary libraries
!pip install transformers torch

# Load model and tokenizer
model_name = "camembert-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Function to generate sentences
def generate_sentence(seed_text):
    inputs = tokenizer(seed_text, return_tensors="pt")
    outputs = model.generate(inputs["input_ids"], max_length=200, num_return_sequences=1)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generating new sentences for each difficulty level
new_sentences = []
for level in data['difficulty']:
    # Randomly select a sentence from the same difficulty level
    seed_text = data[data['difficulty'] == level].sample(1)['sentence'].iloc[0]
    generated_sentence = generate_sentence(seed_text)
    new_sentences.append({'sentence': generated_sentence, 'difficulty': level})

# Append new sentences to the DataFrame
new_df = pd.DataFrame(new_sentences)
augmented_df = pd.concat([data, new_df], ignore_index=True)

# Saving the augmented DataFrame
# augmented_df.to_csv('augmented_dataset.csv', index=False)

(14400, 2)

In [ ]:
augmented_df = pd.read_csv('https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/augmented_dataset.csv')

In [ ]:
def remove_duplicate_words(sentence):
  '''
  Function necessary to remove duplicated words in sentences generated by gpt-2 that made no sense.
  '''
    words = sentence.split()
    seen = set()
    unique_words = []
    for word in words:
        if word not in seen:
            unique_words.append(word)
            seen.add(word)
    return ' '.join(unique_words)

# Apply the function to the 'sentence' column
augmented_df.loc[4800:, 'sentence'] = augmented_df.loc[4800:, 'sentence'].apply(remove_duplicate_words)

In [ ]:
# Function to fix the 'id' column
def fix_id_column(df):
    # Create a sequence of row numbers starting from 0
    correct_ids = pd.Series(range(len(df)))
    # Replace 'NaN' values and incorrect ids
    df['id'] = correct_ids
    return df

# Fix the 'id' column
augmented_df = fix_id_column(augmented_df)

In [ ]:
import nltk
from nltk.corpus import wordnet
from random import randint
import random

nltk.download('wordnet')
nltk.download('omw-1.4')

def synonym_replacement(sentence, n):
    words = sentence.split()
    new_words = words.copy()
    random_word_list = list(set([word for word in words if wordnet.synsets(word)]))
    num_replaced = 0

    for random_word in random_word_list:
        synonyms = set()
        for syn in wordnet.synsets(random_word):
            for lemma in syn.lemmas():
                synonym = lemma.name().replace("_", " ").replace("-", " ").lower()
                synonyms.add(synonym)
        if len(synonyms) > 1:
            synonyms.discard(random_word)
            synonym = list(synonyms)[randint(0, len(synonyms) - 1)]
            new_words = [synonym if word == random_word else word for word in new_words]
            num_replaced += 1
        if num_replaced >= n:  # only replace up to n words
            break

    return ' '.join(new_words)

def shuffle_sentence(sentence):
    words = sentence.split()
    random.shuffle(words)
    return ' '.join(words)


augmented_data = []

# Augment data
for _, row in augmented_df.iterrows():
    original_sentence = row['sentence']
    difficulty = row['difficulty']

    # Generate augmented sentences
    augmented_sentence_synonym = synonym_replacement(original_sentence, 2)
    augmented_sentence_shuffled = shuffle_sentence(original_sentence) #idk


    # Append original and augmented sentences to the new list
    augmented_data.append({'sentence': original_sentence, 'difficulty': difficulty})
    augmented_data.append({'sentence': augmented_sentence_synonym, 'difficulty': difficulty})
    augmented_data.append({'sentence': augmented_sentence_shuffled, 'difficulty': difficulty})


# Create a new DataFrame from the augmented data
data = pd.DataFrame(augmented_data)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
# Preprocess the data: encode labels
label_encoder = LabelEncoder()
data['Label'] = label_encoder.fit_transform(data['difficulty'])

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(data['sentence'], data['Label'], test_size=0.05, random_state=42)

In [ ]:
from transformers import CamembertTokenizer, CamembertModel
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Load tokenizer and model
tokenizer = CamembertTokenizer.from_pretrained("camembert-base")
model = CamembertModel.from_pretrained("camembert-base").to(device)


Using device: cuda


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
def get_embeddings(sentences):
    # Ensure sentences are in a list and filter out any non-string entries
    sentences = [s for s in sentences if isinstance(s, str)]

    # Check if the batch is empty after filtering
    if not sentences:
        return np.array([])  # Return an empty array if no valid sentences are present

    # Tokenize and encode the batch of sentences
    inputs = tokenizer(sentences, return_tensors="pt", padding=True, truncation=True, max_length=512).to(device)

    # Get embeddings
    with torch.no_grad():
        outputs = model(**inputs)

    # Extract the embeddings from the last hidden state
    embeddings = outputs.last_hidden_state[:, 0, :].squeeze().cpu().numpy()
    return embeddings

# Batch processing of embeddings
batch_size = 8  # Adjust batch size according to your GPU memory
X_train_embeddings = np.vstack([get_embeddings(X_train[i:i + batch_size]) for i in range(0, len(X_train), batch_size) if len(X_train[i:i + batch_size]) > 0])
X_test_embeddings = np.vstack([get_embeddings(X_test[i:i + batch_size]) for i in range(0, len(X_test), batch_size) if len(X_test[i:i + batch_size]) > 0])


In [ ]:
from sklearn import svm
clf = svm.SVC(kernel='linear')
clf.fit(X_train_embeddings, y_train)

# Predict on the test set
y_pred = clf.predict(X_test_embeddings)


In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred.round())
print("Accuracy:", accuracy)

Accuracy: 0.6208333333333333


In [ ]:
unlabelled_url = "https://raw.githubusercontent.com/GiammarcoBozzelli/DSML/main/DATA/unlabelled_test_data.csv"
unlabelled_data = pd.read_csv(unlabelled_url)

In [ ]:
unlabelled_embeddings = np.vstack([get_embeddings(unlabelled_data['sentence'][i:i + batch_size]) for i in range(0, len(unlabelled_data), batch_size) if len(unlabelled_data['sentence'][i:i + batch_size]) > 0])

In [ ]:
predicted_labels = clf.predict(unlabelled_embeddings)

# Decode the predicted labels back to original labels
decoded_labels = label_encoder.inverse_transform(predicted_labels)

In [ ]:
submission = pd.DataFrame({'id': unlabelled_data['id'], 'difficulty': decoded_labels})

submission.to_csv('sub.csv', index=False)